# Tarea 3 
### Implementar LDA y LDAseq a tavés de Gensim

Fernanda Weiss 
201373536-3

## Parte 1: Corpus

En el presente trabajo se trabajará con Gensim, una colección de varios scripts diseñados en Python, con el fin de trabajar con grandes colecciones de texto, usando streaming de datos y algoritmos incrementales eficientes, siendo esta la gran diferenciación con el resto de los paquetes científicos. 

Gensim será utilizado con  un corpus preprocesado de noticias recogidas durante marzo, abril y mayo del 2016, tomando como tópicos política, deporte, negocios, tecnología y entretenimiento.


In [1]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
import pyLDAvis

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-10 22:03:35.547947. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [2]:
#Cargar el corpus y su respectivo diccionario en python.
corpus = bleicorpus.BleiCorpus('Tarea3/corpus_lda/corpus_lda.lda_c')
dictionary = Dictionary.load('Tarea3/corpus_lda/corpus_lda.dict')

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-10 22:03:38.781976. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Parte 2: LDA

[//]: <> (Primera pregunta)
Latent Dirichlet Allocation (LDA) es un modelo generativo probabilístico de un corpus, en donde se tiene que cada documento del corpus es representado como una mezcla de tópicos latentes. Estos tópicos hacen referencia a los temas que se tratan en el corpus, y el hecho que se denominen latentes es debido a que no son observados directamente en los documentos ni en el corpus. Además cada uno de estos tópicos está conformado por una distribución sobre la palabras del vocabulario.
El modelo de LDA trabaja el proceso generativo de la siguiente forma para cada documento *d* en el corpus:
1. El largo *N* del documento *d* se comporta como una distribución de Poisson: $N$ ~ $Poisson(𝜉)$
2. Los tópicos $\theta$ del documento *d* se comportan como una distribución de Dirichlet, que indica la proporción de los tópicos en *d*: $\theta$~$Dirichlet(\alpha)$  
3. Para cada palabra $w_i$ del documento *d*:
   1. Tomar un tópico $z_i$ que se comporta como una distribución Multinomial de todos los tópicos de d: $z_i$~$Multinomial(\theta)$
   2. Se tiene la distribución $\phi$ que se comporta como una distribución de Dirichlet de \beta sobre el tópico $z_i$: $\phi$~$Dirichlet(\beta_{z_i})$
   3. Toma la palabra $w_i$ que se comporta como una distribución Multinomial: $w_i$~$Multinomial(\phi)$

Del modelo generativo es posible notar que se LDA tiene como parámetros del proceso a $z_i$ y $\theta$ . Sin embargo, se necesita de otros parámetros, $\alpha$ y $\beta$, llamado hiper-parámetros para poder obtener los parámetro propios de LDA. Es importante mencionar que para estos hiper-parámetros se asume simetría es decir que para todo el modelo generativo $\alpha$ y $\beta$ serán los mismos y positivos.

![imagen1 lda](lda.png)

La Imagen 1 es la representación de LDA mediante plate notation (notación de plato), que muestra el proceso generativo descrito anteriormente. 
Cada círculo representa una variable aleatoria, según su distribución de probabilidad correspondiente, excepto para los hiper-parámetros $\alpha$ y $\beta$ que son valores positivos y constantes para toda la ejecución de LDA que deben ser definidos previamente:
* $\theta$~$Dirichlet(\alpha)$
* $\phi$~$Dirichlet(\beta_{z_i})$ 
* $z_i$~$Multinomial(\theta)$ 
* $w_i$~$Multinomial(\phi)$

Destacar que $w_i$ es la variable aleatoria diferente a las demás debido a que es la única observada.

Luego, la forma en que se va ejecutando el proceso está determinado por la dirección de las conexiones entre los círculos, es decir, el origen de la flecha condiciona el resultado, que es el fin de la flecha. Entonces, por ejemplo, según el diagrama de $\alpha$ se obtiene la variable aleatoria $\theta$, lo cual se condice claramente con lo explicado anteriormente en el punto 2.
Una vez que se entiende el orden lógico que sigue el diagrama, es importante saber que significan los platos (cajas rectangulares), de donde proviene el nombre de la representación. Cada plato indica que la sección que encierra debe ser ejecutada repetidamente, y la cantidad de veces que debe repetirse está indicada en la esquina inferior derecha de cada plato. Para LDA la cantidad de ejecuciones representan:
* *K*: Cantidad de tópicos a observar 
* *N*: Cantidad de palabras en todos los documentos
* *M*: Cantidad de documentos en el corpus 
A través de esta representación es fácil visualizar los hiper-parámetros, parámetros, resultado y orden que se ejecuta LDA.


[//]: <> (Segunda pregunta)
Tal como se mencionó anteriormente, $\alpha$ y $\beta$ son hiper-parámetros que permiten escoger los parámetros propios de LDA. Por un lado, el hiper-parámetro $\alpha$ condiciona la distribución de los tópicos para un documento *d*, en donde al tener un valor de $\alpha$ pequeño, $\alpha < 1$, lo que produce es realizar sesgos por tópico, es decir, que los documentos están dominados por algunos pocos tópicos, por lo que sirve para trabajar con corpus de temas específicos. En el caso que $\alpha$ sea igual a 1, lo que se logra es uniformidad sobre los tópicos. Ahora, si $\alpha$ toma un valor alto, $\alpha > 1$, lo que produce es una suavización de los tópicos, es decir, que los documentos están compuesto por una mezcla de la mayoría o muchos de tópicos, por lo que sirve para trabajar con corpus de temas variados.
Por otro lado, el hiper-parámetro $\beta$ condiciona la distribución de las palabras en los tópicos. Análogamente a $\alpha$, para valores bajos de $\beta$, produce sesgos por palabras dentro de los tópicos, es decir que un tópico puede estar compuesto por solo algunas palabras. Mientras que para valores altos de $\beta$, se produce una suavización de las palabras dentro de los tópicos, es decir que un tópico estará compuesto por una mezcla de la mayoría de las palabras.


[//]: <> (Tercera pregunta)
A continuación, se entrena el corpus usando LDA usando con la siguiente cantidad de tópicos k = 3, 5 y 10. Además se agrega el parámetro *id2word = dictionary* lo que parea el id de la palabra con la palabra en sí, para que luego al visualizar la distribución de palabras en el tópico se muestre la palabra y no su id. 

Se podrían fijar los hiper-parámetros $\alpha$ y $\beta$, sin embargo, para esta ejecución se dejarán por defecto, ambos simétricos (constantes en toda la ejecución) con valor $\frac{1}{num\_topics}$.

Para no repetir el entrenamiento, se guardan los resultados entregados en documentos externos, y además se almacenan en variables distintas.

In [3]:
#pregunta 3
lda3 = ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary)
lda3.save("res_lda/lda_3/lda_3")

lda5 = ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary)
lda5.save("res_lda/lda_5/lda_5")

lda10 = ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary)
lda10.save("res_lda/lda_10/lda_10")

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-10 22:03:42.889068. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


#  ¿Qué información puede desprender de cada tópico? ¿Tienen las palabras descriptivas de cada tópico coherencia con los documentos pertenecientes al corpus? ¿Cómo afecta la cantidad definida de tópicos a la distribución de estos ?



[//]: <> (Cuarta pregunta)
Luego, para cada modelo se muestran los tópicos con su respectiva distribución de palabras, mostrando solo las primeras 20 palabras, en vez de las 10 que se visualizarían por defecto.

Para efecto de analizar los resultados se tratarán como "palabras claves" aquellas palabras que logren describir con certeza un tema, como que por ejemplo *government* está relacionado con el tema de *politica*.

In [4]:
num_words = 20 #por defecto son 10

lda3.print_topics(3, num_words)

[(0,
  u'0.004*"could" + 0.003*"time" + 0.003*"game" + 0.003*"government" + 0.003*"years" + 0.002*"last" + 0.002*"first" + 0.002*"two" + 0.002*"bbc" + 0.002*"get" + 0.002*"take" + 0.002*"still" + 0.002*"made" + 0.002*"think" + 0.002*"many" + 0.002*"good" + 0.002*"added" + 0.002*"world" + 0.002*"football" + 0.002*"make"'),
 (1,
  u'0.003*"years" + 0.003*"time" + 0.003*"could" + 0.003*"last" + 0.002*"get" + 0.002*"government" + 0.002*"two" + 0.002*"first" + 0.002*"like" + 0.002*"make" + 0.002*"next" + 0.002*"blair" + 0.002*"three" + 0.002*"home" + 0.002*"many" + 0.002*"minister" + 0.002*"way" + 0.002*"bbc" + 0.002*"well" + 0.002*"election"'),
 (2,
  u'0.004*"last" + 0.003*"could" + 0.003*"first" + 0.003*"years" + 0.003*"made" + 0.003*"two" + 0.002*"film" + 0.002*"government" + 0.002*"number" + 0.002*"world" + 0.002*"make" + 0.002*"music" + 0.002*"many" + 0.002*"back" + 0.002*"top" + 0.002*"three" + 0.002*"time" + 0.002*"game" + 0.002*"well" + 0.002*"like"')]

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-10 22:05:13.095352. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


Para el primer entrenamiento, es posible visualizar los 3 tópicos (0, 1 y 2). De estos, es notoria la baja proporción de cada palabra en cada tópicos. Es más, de los 3 tópicos, la palabra con mayor probabilidad es *could* y *last* con 0.004 en el primer y tercer tópico respectivamente, sin ser estás palabras descriptivasal momento de análisis. 

Es posible notar que ningún tópico entrega información realmente descriptiva sobre el tema que se esta hablando, es decir, los resultados no son buenos. Teniendo esto en cuenta, se podría mencionar que:
* En el primer tópico está mezclado entre deporte y política, debido a que dentro de las palabras con mayor proporción están por un lado *game* y *football*, y por otro *government*, que son consideradas palabras claves dentro de cada tema.
* En el segundo tópico se podría decir que el tópico trata sobre política dado que tiene palabras como *government*, *first*, *blair*, *minister* y *election*, pues Tony Blair fue Primer Ministro de Reino Unido.
* En el tercer tópico se podría decir que está mayormente relacionado con entretenimiento, dado que tiene palabras claves como *film*, *music* y *game*. Sin embargo, tiene palabras de otros temas como política con *government* o deporte si es que se considera *game* de este tema.



In [5]:
lda5.print_topics(5, num_words)

[(0,
  u'0.004*"could" + 0.004*"two" + 0.003*"first" + 0.003*"last" + 0.003*"make" + 0.002*"time" + 0.002*"years" + 0.002*"bbc" + 0.002*"like" + 0.002*"well" + 0.002*"world" + 0.002*"three" + 0.002*"number" + 0.002*"games" + 0.002*"says" + 0.002*"game" + 0.002*"may" + 0.002*"government" + 0.002*"players" + 0.002*"news"'),
 (1,
  u'0.004*"could" + 0.003*"years" + 0.003*"government" + 0.003*"world" + 0.003*"may" + 0.002*"two" + 0.002*"first" + 0.002*"get" + 0.002*"many" + 0.002*"time" + 0.002*"bbc" + 0.002*"last" + 0.002*"made" + 0.002*"way" + 0.002*"say" + 0.002*"game" + 0.002*"film" + 0.002*"make" + 0.002*"good" + 0.002*"back"'),
 (2,
  u'0.004*"first" + 0.003*"government" + 0.003*"last" + 0.003*"make" + 0.002*"many" + 0.002*"show" + 0.002*"years" + 0.002*"could" + 0.002*"get" + 0.002*"bbc" + 0.002*"made" + 0.002*"film" + 0.002*"back" + 0.002*"added" + 0.002*"time" + 0.002*"well" + 0.002*"two" + 0.002*"want" + 0.002*"public" + 0.002*"music"'),
 (3,
  u'0.004*"last" + 0.004*"could" + 0.

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-10 22:05:17.124412. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


Para el segundo entrenamiento, es posible visualizar los 5 tópicos (0, 1, 2, 3 y 4) encontrados, con sus respectivas distribución de palabras. Al igual que en el primer entrenamiento es posible observar la baja proporción de cada palabra en cada tópico.

A pesar que se esperaría que cada tópico hable sobre un tema del corpus, es posible observar que la información entregada por cada tópico no es muy descriptiva, al igual que en el caso anterior, con 3 tópicos. En general, las palabras mostradas con mayor proporción no son muy informativas, por lo que no queda claro que es lo que se intenta representar en cada tópico. Sin embargo, de cada tópico se podría decir que:
* Primer tópico: Trataría mayormente de deporte, debido a las palabras *games*, *game* y *players* que aparecen como "palabras claves", aunque la palabra *government* puede inducir a una relación entre el deporte y la política.
* Segundo tópico: No es posible decir con cierta seguridad de que trata el tópico, dado que tiene "palabras claves" como *government* relacionado con politica, *film* relacionado con entretención y *game* relacionado con deporte o entretención.
* Tercer tópico: Se podría decir que el tópico habla de entretemiento por contener palabras como *film* o *music*, pero también estaría relacionado con política debido a *government*.
* Cuarto tópico: Al igual que en tópico anterior, se podría decir que el tópico está relacionado con entreteniemitno dado que tiene "palabras claves" como *film*, *music* y *game*. Aunque también estaría relacionado con política debido a *government*.
* Quinto tópico: Se podría pensar que el tópico habla de política dado que tiene palabras como *blair*, *first* y *minister*, pues Tony Blair fue Primer Ministro de Reino Unido.

In [11]:
lda10.print_topics(10, num_words)

[(0,
  u'0.004*"could" + 0.004*"last" + 0.003*"first" + 0.003*"government" + 0.003*"years" + 0.003*"says" + 0.002*"get" + 0.002*"like" + 0.002*"make" + 0.002*"way" + 0.002*"bbc" + 0.002*"time" + 0.002*"blair" + 0.002*"work" + 0.002*"well" + 0.002*"sales" + 0.002*"users" + 0.002*"market" + 0.002*"games" + 0.002*"good"'),
 (1,
  u'0.004*"years" + 0.004*"time" + 0.004*"last" + 0.003*"game" + 0.003*"could" + 0.003*"make" + 0.003*"first" + 0.003*"world" + 0.003*"club" + 0.002*"think" + 0.002*"league" + 0.002*"two" + 0.002*"good" + 0.002*"added" + 0.002*"growth" + 0.002*"economy" + 0.002*"many" + 0.002*"may" + 0.002*"three" + 0.002*"liverpool"'),
 (2,
  u'0.004*"could" + 0.004*"game" + 0.004*"government" + 0.003*"last" + 0.003*"get" + 0.003*"first" + 0.002*"made" + 0.002*"time" + 0.002*"number" + 0.002*"back" + 0.002*"three" + 0.002*"two" + 0.002*"show" + 0.002*"club" + 0.002*"group" + 0.002*"may" + 0.002*"united" + 0.002*"top" + 0.002*"next" + 0.002*"make"'),
 (3,
  u'0.004*"first" + 0.003*

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 00:29:23.741746. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


Para el tercer entrenamiento, es posible visualizar los 10 tópicos (0, 1, 2,..., 9) encontrados, con sus respectivas distribución de palabras. Al igual que en los entrenamiento anteriores es posible observar la baja proporción de cada palabra en cada tópico.

Es posible observar que la mayor parte de la información entregada por cada tópico no es muy descriptiva, al igual que en los casos anterior. En general, las palabras mostradas con mayor proporción no son muy informativas, por lo que no queda claro que es lo que se intenta representar en cada tópico. A pesar de esto,  se podría decir de cada tópico que:

* Primer tópico: Es un tópico bien variado, que contiene información sobre política debido a la aparición de las palabras *government* y *blair*, además contiene información sobre negocios por palabras como *work*, *market* y *sales*, y también se podría pensar que se está hablando de tecnología por la palabra *users* y de entretención o deporte por *games*. 
* Segundo tópico: En este tópico se estaría hablando mayormente sobre deporte, por palabras como *game*, *club*, *league* y *liverpool*, pero también estaría relacionado con negocios o política por la palabra *economy*. 
* Tercer tópico: Este tópico también tendría más presente el tema deporte (*game*, *club*) pero no tan marcadamente como en el tópico anterior.         
* Cuarto tópico: El tópico tendría más presente el tema político debido a palabras clave como *minister* y *government*, a pesar que estas aparezcan con menor probabiliad que las otras palabras, pues ellas no son informativas.
* Quinto tópico: film government game 
* Sexto tópico: governmen film music award
* Septimo tópico: music band album british club film -> claramente entrtencion
* Octavo tópico: blair election games party
* Noveno tópico: film government law lord music minister -> politica con un poco de entretención xd
* Décimo tópico: government game company security minister

Se esperaria que el topico 5 es mejor y cada uno represente un tema ...
10 topicos aparecen nuevas palabras 

## Parte 3: Dynamic Topic Models

Dynamic Topic Models (DTM) también es un modelo generativo extendido de LDA, pero que a diferencia de éste permite analizar la evolución de los tópicos en un corpus a través del tiempo.

[//]: <> (Primera pregunta)
Primero se procede a entrenar nuevamente el corpus entregado, pero esta vez utilizando el algoritmo LdaSeq, el cual está basado en el código orginal de DTM de Blei. Para el entrenamiento se utilizarán 5 tópicos con una varianza *0.001*, y además se debe entregar como parámetro la cantidad de documentos por intervalo de tiempo, que en este caso serían los documentos para cada mes, con 3 intervalos de tiempo, dado que los documentos pertenecen a 3 meses (marzo, abril y mayo): 438 documentos para el primer intervalo, 430 para e segundo intervalo y 456 para el último intervalo.

Al igual que el LDA, se introduce el parámetro *id2word=dictionary* en la ejecución, para realizar el pareo entre el id de las palabras del diccionario y las palabras.

In [23]:
from gensim.models import ldaseqmodel

corpus = bleicorpus.BleiCorpus('Tarea3/corpus_ldaseq/corpus_ldaseq.lda_c')
dictionary = Dictionary.load('Tarea3/corpus_ldaseq/corpus_ldaseq.dict')

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 17:28:17.597023. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [24]:
time_slice = [438, 430, 456]
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=5, chain_variance = 0.001)
ldaseq.save("res_ldaseq/ldaseq")

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 17:29:21.820119. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


[//]: <> (Segunda pregunta Para cada uno de los intervalos de tiempo, observe la distribución topic_word de cada tópico ¿Que puede observar en la evolución de estos?)

A continuación se muestra el resultado de obtenido de LdaSeq, el cual entrega todos los tópicos en un intervalo de tiempo específico, dado que la ejecución solo se realizó con 5 tópicos, la únca variación es el intervalo de tiempo a visualizar. La cantidad de palabras a mostrar para cada tópico serán las por defecto, 20.

En el primer intervalo, que se asume el marzo de 2016 se obtiene la siguiente distribución de palabras para cada tópico:

In [25]:
ldaseq.print_topics(0)

[[(u'government', 0.007),
  (u'blair', 0.006),
  (u'film', 0.006),
  (u'labour', 0.006),
  (u'minister', 0.005),
  (u'election', 0.005),
  (u'party', 0.004),
  (u'public', 0.004),
  (u'bbc', 0.004),
  (u'prime', 0.003),
  (u'brown', 0.003),
  (u'could', 0.003),
  (u'howard', 0.003),
  (u'lord', 0.003),
  (u'plans', 0.003),
  (u'say', 0.003),
  (u'law', 0.003),
  (u'secretary', 0.003),
  (u'britain', 0.003),
  (u'british', 0.003)],
 [(u'music', 0.011),
  (u'show', 0.008),
  (u'years', 0.006),
  (u'number', 0.006),
  (u'top', 0.006),
  (u'last', 0.006),
  (u'band', 0.005),
  (u'album', 0.005),
  (u'games', 0.005),
  (u'awards', 0.005),
  (u'first', 0.005),
  (u'song', 0.004),
  (u'game', 0.004),
  (u'three', 0.004),
  (u'star', 0.004),
  (u'award', 0.004),
  (u'two', 0.004),
  (u'chart', 0.003),
  (u'record', 0.003),
  (u'rock', 0.003)],
 [(u'use', 0.007),
  (u'technology', 0.007),
  (u'users', 0.006),
  (u'could', 0.006),
  (u'many', 0.006),
  (u'net', 0.005),
  (u'online', 0.005),
  (u

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 19:02:13.921426. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


En el segundo intervalo, que se asume el abril de 2016 se obtiene la siguiente distribución de palabras para cada tópico:

In [26]:
ldaseq.print_topics(1)

[[(u'government', 0.007),
  (u'blair', 0.006),
  (u'film', 0.006),
  (u'labour', 0.006),
  (u'minister', 0.005),
  (u'election', 0.005),
  (u'party', 0.004),
  (u'public', 0.004),
  (u'brown', 0.004),
  (u'bbc', 0.004),
  (u'prime', 0.003),
  (u'could', 0.003),
  (u'howard', 0.003),
  (u'lord', 0.003),
  (u'plans', 0.003),
  (u'say', 0.003),
  (u'law', 0.003),
  (u'secretary', 0.003),
  (u'britain', 0.003),
  (u'british', 0.003)],
 [(u'music', 0.011),
  (u'show', 0.008),
  (u'years', 0.006),
  (u'number', 0.006),
  (u'top', 0.006),
  (u'last', 0.006),
  (u'band', 0.005),
  (u'album', 0.005),
  (u'awards', 0.005),
  (u'first', 0.005),
  (u'games', 0.005),
  (u'song', 0.005),
  (u'game', 0.004),
  (u'three', 0.004),
  (u'star', 0.004),
  (u'award', 0.004),
  (u'two', 0.004),
  (u'chart', 0.003),
  (u'record', 0.003),
  (u'rock', 0.003)],
 [(u'use', 0.007),
  (u'technology', 0.007),
  (u'users', 0.006),
  (u'could', 0.006),
  (u'many', 0.006),
  (u'online', 0.005),
  (u'net', 0.005),
  (u

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 19:02:19.376440. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


En el último intervalo, que se asume el mayo de 2016 se obtiene la siguiente distribución de palabras para cada tópico:

In [27]:
ldaseq.print_topics(2)

[[(u'government', 0.007),
  (u'blair', 0.006),
  (u'film', 0.006),
  (u'labour', 0.006),
  (u'minister', 0.005),
  (u'election', 0.005),
  (u'party', 0.004),
  (u'public', 0.004),
  (u'brown', 0.004),
  (u'bbc', 0.004),
  (u'prime', 0.003),
  (u'could', 0.003),
  (u'howard', 0.003),
  (u'lord', 0.003),
  (u'plans', 0.003),
  (u'say', 0.003),
  (u'law', 0.003),
  (u'secretary', 0.003),
  (u'britain', 0.003),
  (u'british', 0.003)],
 [(u'music', 0.011),
  (u'show', 0.008),
  (u'years', 0.006),
  (u'number', 0.006),
  (u'last', 0.006),
  (u'top', 0.006),
  (u'band', 0.005),
  (u'album', 0.005),
  (u'awards', 0.005),
  (u'first', 0.005),
  (u'games', 0.005),
  (u'song', 0.005),
  (u'game', 0.004),
  (u'three', 0.004),
  (u'award', 0.004),
  (u'star', 0.004),
  (u'two', 0.004),
  (u'chart', 0.003),
  (u'record', 0.003),
  (u'rock', 0.003)],
 [(u'use', 0.007),
  (u'technology', 0.007),
  (u'users', 0.006),
  (u'could', 0.006),
  (u'many', 0.006),
  (u'online', 0.005),
  (u'information', 0.00

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 19:02:25.228077. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


[//]: <> (Tercera pregunta Para un documento dado del corpus , obtenga la distribución de tópicos usando el modelo entrenado y visualice el contenido de dicho documento. Analice sus observaciones. Repita el experimento con los 3 modelos generados.)
asd

In [28]:
words = [dictionary[word_id] for word_id, count in corpus[558]]
print (words)
doc_bow = corpus[558]  
print (ldaseq[doc_bow])

[u'held', u'set', u'away', u'taking', u'number', u'position', u'money', u'lord', u'industry', u'top', u'made', u'could', u'end', u'week', u'return', u'ended', u'second', u'box', u'final', u'took', u'broke', u'last', u'bill', u'taken', u'series', u'according', u'figures', u'provide', u'place', u'records', u'films', u'film', u'christmas', u'major', u'paul', u'title', u'much', u'years', u'king', u'help', u'however', u'estimates', u'competition', u'falling', u'produce', u'third', u'weve', u'robert', u'days', u'north', u'high', u'helped', u'success', u'stronger', u'history', u'ben', u'making', u'office', u'increase', u'cast', u'american', u'president', u'howard', u'meet', u'crown', u'rings', u'spot', u'parents', u'alone', u'weekend', u'day', u'record', u'events', u'comedy', u'starring', u'finished', u'chart', u'overall', u'previous', u'highest', u'hughes', u'stars', u'fourth', u'aviator', u'moved', u'sequel', u'studio', u'releases', u'revenue', u'industrys', u'dicaprio', u'leonardo', u'note

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 19:02:46.201315. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


[//]: <> (Cuarta pregunta Instale el módulo pyLDAvis para visualizar el modelo generado por LDAseq se recomienda usar el paquete pip para instalar pyLDAvis. Genere una visualización para cada intervalo de tiempo modifique el parámetro time)  

A continuación mediante el módulo pyLDAvis se genera una visualización para cada intervalo de tiempo.

Para el primer intervalo de tiempo (marzo) se obtiene:

In [29]:
doc_topic, topic_term, doc_lengths, term_frequency, vocab = ldaseq.dtm_vis(time=0, corpus=corpus)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_dtm)

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 19:02:59.888154. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


Para el segundo intervalo de tiempo (abril) se obtiene:

In [30]:
doc_topic, topic_term, doc_lengths, term_frequency, vocab = ldaseq.dtm_vis(time=1, corpus=corpus)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_dtm)

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 19:03:09.977002. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


Para el último intervalo de tiempo (mayo) se obtiene:

In [31]:
doc_topic, topic_term, doc_lengths, term_frequency, vocab = ldaseq.dtm_vis(time=2, corpus=corpus)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_dtm)

/usr/lib/python2.7/dist-packages/simplejson/encoder.py:275: DeprecationWarning: Interpreting naive datetime as local 2017-06-11 19:03:15.593688. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


[//]: <> (quinta pregunta Compare los resultados obtenidos con el modelo LDA de la sección anterior, con los resultados de ldaseq ¿Que modelo logra representar mejor el corpus de documentos usado? ¿Como influye en el entrenamiento de los tópicos la consideración de los intervalos de tiempo?.)

Al comparar los resultados entregados por LDA y LdaSeq es posible determinar que para este corpus son mejores los resultados entregados por el algoritmo de LdaSeq.